In [1]:
import os
import json
import pandas as pd
import copy

In [2]:
exfname1="../../experiment/data/AllCodesNew.csv"
outfname="../../experiment/data/AllCodesNew.csv"
path_name="../../experiment/data/raw_dataExp1"

In [3]:
fnames=[]
entries={}
numWorkers={}
workerSessions={}
sessionCount={}
sessionToExpID={}
allData={}
total=0
for dirpath, dirnames, filenames in os.walk("./"+path_name+"/results/"):
    for filename in [f for f in filenames if f.endswith("PT.json")]:
        fname=os.path.join(dirpath, filename)
        with open(fname, 'r') as f:
            parsed_json = json.load(f)
        if parsed_json['sid'] not in entries:
            entries[parsed_json['sid']]=0
            sessionCount[parsed_json['sid']]=0
        entries[parsed_json['sid']]+=float(parsed_json['tspan'])
        sessionCount[parsed_json['sid']]+=1
        total+=1
        
        if parsed_json['id'] not in numWorkers:
            numWorkers[parsed_json['id']]=0
            workerSessions[parsed_json['id']]={}
        numWorkers[parsed_json['id']]+=1
        
        if parsed_json['sid'] not in workerSessions[parsed_json['id']]:
            workerSessions[parsed_json['id']][parsed_json['sid']]=0
        workerSessions[parsed_json['id']][parsed_json['sid']]+=1
        
        #workerSessions[parsed_json['id']].append(parsed_json['sid'])
        #print(json.dumps(parsed_json, indent=4, sort_keys=True))
        #print(os.path.join(dirpath, filename))

sessionToExpID={}
for dirpath, dirnames, filenames in os.walk("./"+path_name+"/results/"):
    for filename in [f for f in filenames if f.endswith("header.json")]:
        fname=os.path.join(dirpath, filename)
        with open(fname, 'r') as f:
            parsed_json = json.load(f)
            #print(json.dumps(parsed_json, indent=4, sort_keys=True))
        sessionToExpID[parsed_json['sid']]=parsed_json['eid']
        allData[parsed_json['sid']]=parsed_json

In [4]:
if len(numWorkers)!=len(entries):
    print("There is an issue check it",len(numWorkers),len(entries))

In [5]:
# Taking only the best 
#for worker,val in workerSessions.items():
#    if len(val) > 1:
#        print("Worker {} performed the expriment {} times, keeping only the best one".format(worker,len(val)))
#        best=0
#        keepsid=0
#        for sid, num in val.items():
#            if num>best:
#                best=num
#                keepsid=sid
#        for sid, num in val.items():
#            if sid!=keepsid:
#                del sessionToExpID[sid]

In [6]:
total=0
for dirpath, dirnames, filenames in os.walk("./"+path_name+"/results/"):
    for filename in [f for f in filenames if f.endswith(".csv")]:
        total+=1

In [7]:
df=pd.read_csv(exfname1)
ncolumns=['State','TrialCount','Bonus','Time','Age','Gender','WorkerID','SessionID','Group','Date']
df=pd.concat([df,pd.DataFrame(columns=ncolumns)],sort=False)
len(df)

540

In [8]:
#df.head()

In [9]:
#bonus={}
#count=0
#for sessionID,time in entries.items():
#    count+=1
    #print(count,sessionToExpID[sessionID],time,sessionCount[sessionID])

In [10]:
def ModifyRow(row,val):
    row.loc['State']=val['State']
    row.loc['TrialCount']=val['tcount']
    row.loc['Bonus']=val['Bonus']
    row.loc['Time']=val['sumTime']
    row.loc['Age']=val['age']
    row.loc['Gender']=val['ge']
    row.loc['WorkerID']=val['id']
    row.loc['SessionID']=val['sid']
    row.loc['Group']=val['gr']
    row.loc['Date']=val['da']
    return row

In [11]:
time_limit=20*60
print(time_limit)

seenExpID={}
count=0
for sessionID in sessionToExpID.keys() & allData.keys():
    expID=sessionToExpID[sessionID]
    count+=1
    if sessionID not in entries:
        time=-1
        trialCount=0
    else:
        time=entries[sessionID]
        trialCount=sessionCount[sessionID]
    allData[sessionID]['tcount']=trialCount
    allData[sessionID]['sumTime']=time
    if trialCount==6:
        allData[sessionID]['State']='Valid'
        if time < time_limit:
            allData[sessionID]['Bonus']='True'                   
        else:
            allData[sessionID]['Bonus']='False'                   
    else:
        allData[sessionID]['State']='Invalid'
        allData[sessionID]['Bonus']='False'
    
    if expID not in seenExpID:
        seenExpID[expID]=0
        if expID in df['ExpID'].values:
            if len(df[df['ExpID']==expID])==1:
                row=ModifyRow(df.loc[df['ExpID']==expID].iloc[0],allData[sessionID])
                df.loc[df['ExpID']==expID,:]=row.values
                pass
            else:
                print("0 ERROR REPEATED ROW this should not happen ",(len(df[df['ExpID']==expID])))
        else:
            print("0 ERROR Not exiting ExpID this should not happen ",expID)
    else:
        if expID in df['ExpID'].values:
            if len(df[df['ExpID']==expID])==1:
                pastTrialCount=df.loc[df['ExpID']==expID,'TrialCount'].values[0]
                pastWorkerID=df.loc[df['ExpID']==expID,'WorkerID'].values[0]
                #if pastWorkerID == allData[sessionID]['id']:
                if pastTrialCount<allData[sessionID]['tcount']:
                    row=ModifyRow(df.loc[df['ExpID']==expID].iloc[0],allData[sessionID])
                    df.loc[df['ExpID']==expID,:]=row.values
                else:
                    print("{} Ignoring Worker {}/{} with trials {}/{} new/before".format(expID,allData[sessionID]['id'],pastWorkerID,allData[sessionID]['tcount'],pastTrialCount))
                #else:
                #    print("0 Different Worker Adding Row",expID)
                #    row=ModifyRow(df.loc[df['ExpID']==expID].iloc[0],allData[sessionID])
                #    df=df.append(row, ignore_index=True)
            else:
                row=ModifyRow(df[df['ExpID']==expID].iloc[0],allData[sessionID])
                df=df.append(row, ignore_index=True)
                print("1 ERROR REPEATED ROW this should not happen ",expID)
                print("1 ERROR REPEATED ROW this should not happen",len(df[df['ExpID']==expID].iloc[0,:]))
                #break
        else:
            print("1 ERROR Not exiting ExpID this should not happen ",expID)    

1200
dc9bf4a4673c2bbdce71ea67fb1b7582 Ignoring Worker A2ASTDMWBCFIP0/A2ASTDMWBCFIP0 with trials 0/6 new/before
230631cf661b33dd9919af0eba48599a Ignoring Worker AQZM81DMVPHW5/AQZM81DMNVPHW5 with trials 0/0 new/before
443ac33ac0d74cb1e3de61cc9c839edb Ignoring Worker A3LETROTTA0C7N/A3LETROTTA0C7N with trials 0/0 new/before
1269d856092b0868553963cdc310b55d Ignoring Worker A1HB8WVP9ZWLXV/A1HB8WVP9ZWLXV with trials 0/0 new/before
dc9bf4a4673c2bbdce71ea67fb1b7582 Ignoring Worker A2ASTDMWBCFIP0/A2ASTDMWBCFIP0 with trials 0/6 new/before


In [12]:
df.to_csv("NEW_WITH_BONUS_"+outfname)

In [13]:
fcount=0
fcompleted=0
for sessionID,expID in sessionToExpID.items():
    fcount+=1
    if sessionID not in entries:
        time=-1
        trialCount=0
    else:
        time=entries[sessionID]
        trialCount=sessionCount[sessionID]
    #print(count,expID,time,trialCount)
    if trialCount == 6:
        fcompleted+=1
print("Total completed {}".format(fcompleted))
print("Total count {}".format(fcount))

Total completed 20
Total count 35
